# Submission 2: Deep learning and bias

This document aims to explain how the data was used to draw conclusions, which techniques and models were used to analyze and solve our problem. Our main goal was twofold. We wanted to identify if there is bias in a dataset and how to handle it. The structure of this paper represents our thinking and working process. We began by building several models, then analyzing the fairness of the model under consideration of specific metrics and finally showing two possibilities of how to handle bias in data. Last but not least we describe what implications for businesses this work has. 

Based on the exploratory data analysis in submission 1, we believe four variables:
1. ZIP
2. rent
3. job stability
4. occupation)
cause the bias. 

Thus, our hypothesis is: 

**does removing these four variables reduce bias?**

In [35]:
#import libraries
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.utils import class_weight
from sklearn.utils import resample
from sklearn.model_selection import StratifiedKFold

import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout

In [36]:
#%pip install aif360

In [37]:
from aif360.algorithms.preprocessing import DisparateImpactRemover
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from numba.core.decorators import jit

/Users/Rolf/anaconda3/lib/python3.7/site-packages/aif360/algorithms/preprocessing/lfr_helpers/helpers.py:2: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit
/Users/Rolf/anaconda3/lib/python3.7/site-packages/aif360/algorithms/preprocessing/lfr_helpers/helpers.py:2: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit


In [38]:
#set a random seed
seed = 123
np.random.seed(seed)

### Import train and test dataset

In [39]:
#import pre_processed train dataset
train = pd.read_csv('train_preprocessed.csv', index_col=0)
print('train shape: ' + str(train.shape))
train.head()

train shape: (471136, 13)


,minority,sex,ZIP,rent,education,age,income,loan_size,payment_timing,year,job_stability,occupation,default
0,1,0,1,1,57.230650,36.050927,205168.022244,7600.292199,3.302193,0,3.015554,1,1
1,1,0,1,1,45.891343,59.525251,187530.409981,5534.271289,3.843058,0,5.938132,1,1
2,1,0,1,1,46.775489,67.338108,196912.006690,2009.903438,2.059034,0,2.190777,1,1
3,1,0,1,1,41.784839,24.067401,132911.650615,3112.280893,3.936169,0,1.725860,1,1
4,1,0,1,1,41.744838,47.496605,161162.551205,1372.077093,3.709910,0,0.883104,1,1


In [40]:
#import pre_processed test dataset
test = pd.read_csv('test_preprocessed.csv', index_col=0)
print('test shape: ' + str(test.shape))
test.head()

test shape: (160000, 13)


,minority,sex,ZIP,rent,education,age,income,loan_size,payment_timing,job_stability,year,occupation,default
0,1,0,1,1,51.265723,25.710781,166455.209729,8064.951996,3.874735,43.764963,30,1,1
1,0,0,1,0,58.882849,39.689510,216752.885725,7166.701945,3.809001,46.903977,30,0,0
2,0,0,1,0,56.504545,25.847324,183764.480788,3322.045258,3.497214,63.453467,30,0,0
3,1,0,1,1,47.074111,26.381109,154057.004978,15.223904,3.535370,56.243840,30,1,0
4,1,0,1,1,48.916960,18.779902,143463.038107,7860.534547,3.663330,49.884194,30,1,0


### Modify train and test dataset

**Terminology:**
- Bias dataset = dataset that contains ZIP, rent, job_stability and occupation variables
- Non-Bias dataset = dataset that does **NOT** contain ZIP, rent, jot_stability and occupation variables
- default variable = is our target variables
- minority variable = is our protected group (we're trying to identify bias caused by minority)

**Note**
- We drop sex variable since it is a protected group. Thus, we're not allowed to use sex in our model. Also, we're focusing on bias caused by minority and not sex.

In [41]:
#train dataset
X_tr_nob = train.drop(['default', 'minority', 'rent', 'job_stability', 'occupation', 'sex', 'ZIP'], axis=1)
X_tr_bias = train.drop(['default', 'minority', 'sex'], axis=1)
y_train= train['default']

#test dataset
X_te_nob = test.drop(['default', 'minority', 'rent', 'job_stability', 'occupation', 'sex', 'ZIP'], axis=1)
X_te_bias = test.drop(['default', 'minority', 'sex'], axis=1)
y_test = test['default']

In [42]:
#Scale X train/test Bias
scaler = StandardScaler()
X_fit_bias = scaler.fit(X_tr_bias)

X_train_bias = X_fit_bias.transform(X_tr_bias)
X_test_bias = X_fit_bias.transform(X_te_bias)

#Scale X train/test Non-Bias
X_fit_nob = scaler.fit(X_tr_nob)

X_train_nob = X_fit_nob.transform(X_tr_nob)
X_test_nob = X_fit_nob.transform(X_te_nob)

# 1. Linear classification: Base Model

Linear classification (LogisticRegression) will serve as our base model. We will compare results (accuracy) to deep learning model

In [43]:
#bias
lr_bias = LogisticRegression(solver='lbfgs', random_state=seed)
lr_bias.fit(X_train_bias, y_train)

y_pred_lr_bias = lr_bias.predict(X_test_bias)
accuracy_lr_bias = accuracy_score(y_pred_lr_bias, y_test)


#no bias
lr_nob = LogisticRegression(solver='lbfgs', random_state=seed)
lr_nob.fit(X_train_nob, y_train)

y_pred_lr_nob = lr_nob.predict(X_test_nob)
accuracy_lr_nob = accuracy_score(y_pred_lr_nob, y_test)

#print accuracy
print('Accuracy with bias: ' + str(round(accuracy_lr_bias*100,2)) + '%')
print('Accuracy with no bias: ' + str(round(accuracy_lr_nob*100,2)) + '%')

Accuracy with bias: 37.01%
Accuracy with no bias: 41.49%


In [44]:
#print classification report - BIAS
print('Bias classification report')
print(classification_report(y_test, y_pred_lr_bias))

Bias classification report
              precision    recall  f1-score   support

           0       0.82      0.33      0.47    136055
           1       0.13      0.58      0.22     23945

    accuracy                           0.37    160000
   macro avg       0.48      0.46      0.34    160000
weighted avg       0.72      0.37      0.44    160000



In [45]:
#print classification report - NO BIAS
print('Non-Bias classification report')
print(classification_report(y_test, y_pred_lr_nob))

Non-Bias classification report
              precision    recall  f1-score   support

           0       0.84      0.38      0.53    136055
           1       0.14      0.59      0.23     23945

    accuracy                           0.41    160000
   macro avg       0.49      0.49      0.38    160000
weighted avg       0.74      0.41      0.48    160000



# 2. Deep Learning

In [46]:
import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout

In [47]:
#stop if no improvement in loss after 3 epochs
es_callback = keras.callbacks.EarlyStopping(monitor='loss', patience=5)

#class weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

/Users/Rolf/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass classes=[0 1], y=0       1
1       1
2       1
3       1
4       1
       ..
7995    0
7996    0
7997    0
7998    0
7999    0
Name: default, Length: 471136, dtype: int64 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [48]:
def create_model(df):
    ## Initialize model.
    model = Sequential() #if you want a recurrent NN, then specify here Recurrent()

    ## 1st Layer
    model.add(Dense(64, input_dim=df.shape[1]))
    model.add(Activation('relu'))
    model.add(Dropout(0.5)) #50% of neurons are de-activated randomly per batch. helps with generalisation.

    ## 2nd Layer
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dropout(0.5)) #dropout number is arbitrary. trail and error. 

    ## Adding Softmax Layer
    model.add(Dense(1))
    model.add(Activation('sigmoid')) #softmax used for classification. softmax = [0,1]

    ## Define loss function
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'], weighted_metrics=['accuracy'])

    return model

### Cross-validation

Inspired by the code on this website: 
https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/

In [49]:
#Cross-validation

# define 5-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

def kfold_validation(X, y):

    X_df = pd.DataFrame(X)
    cvscores = []
    
    for train, test in kfold.split(X_df, y):
        model = create_model(X_df)
        model.fit(X_df.iloc[train], y.iloc[train], epochs=20, batch_size=1000, shuffle=True, verbose=0, 
                  callbacks=[es_callback], class_weight=class_weights)
        scores = model.evaluate(X_df.iloc[test], y.iloc[test], verbose=0)
        
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        cvscores.append(scores[1] * 100)
        
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    
    return model

## Model with BIAS

In [50]:
#cross-validaiton
FNN_bias = kfold_validation(X_train_bias, y_train)

accuracy: 99.98%
accuracy: 99.98%
accuracy: 99.98%
accuracy: 99.99%
accuracy: 99.99%
99.98% (+/- 0.00%)


In [51]:
#the kfold model is giving wierd results. 
#thus, switching to this model
#I think issue stems from stratifiedKfold not working, i.e. I'm doing something wrong
FNN_bias = create_model(X_train_bias)
FNN_bias.fit(X_train_bias, y_train, epochs=100, batch_size=1000, shuffle=True, verbose=2, callbacks=[es_callback],
           class_weight=class_weights, validation_data=(X_test_bias, y_test))

Train on 471136 samples, validate on 160000 samples
Epoch 1/100
 - 2s - loss: 0.0793 - accuracy: 0.9705 - accuracy_1: 0.9705 - val_loss: 2.7812 - val_accuracy: 0.4946 - val_accuracy_1: 0.4946
Epoch 2/100
 - 2s - loss: 0.0041 - accuracy: 0.9995 - accuracy_1: 0.9995 - val_loss: 3.5917 - val_accuracy: 0.4848 - val_accuracy_1: 0.4848
Epoch 3/100
 - 2s - loss: 0.0030 - accuracy: 0.9997 - accuracy_1: 0.9997 - val_loss: 4.3423 - val_accuracy: 0.4545 - val_accuracy_1: 0.4545
Epoch 4/100
 - 2s - loss: 0.0026 - accuracy: 0.9997 - accuracy_1: 0.9997 - val_loss: 4.8941 - val_accuracy: 0.4527 - val_accuracy_1: 0.4527
Epoch 5/100
 - 2s - loss: 0.0023 - accuracy: 0.9998 - accuracy_1: 0.9998 - val_loss: 5.0414 - val_accuracy: 0.4590 - val_accuracy_1: 0.4590
Epoch 6/100
 - 2s - loss: 0.0022 - accuracy: 0.9998 - accuracy_1: 0.9998 - val_loss: 5.1433 - val_accuracy: 0.4708 - val_accuracy_1: 0.4708
Epoch 7/100
 - 2s - loss: 0.0021 - accuracy: 0.9998 - accuracy_1: 0.9998 - val_loss: 5.5775 - val_accuracy: 

In [52]:
#predict default with test dataset
y_pred_bias = FNN_bias.predict_classes(X_test_bias)

print('Bias classification report')
print(classification_report(y_test, y_pred_bias))
print()

print('Bias confusion matrix')
print(confusion_matrix(y_test, y_pred_bias))

Bias classification report
              precision    recall  f1-score   support

           0       0.86      0.57      0.69    136055
           1       0.16      0.45      0.23     23945

    accuracy                           0.55    160000
   macro avg       0.51      0.51      0.46    160000
weighted avg       0.75      0.55      0.62    160000


Bias confusion matrix
[[77931 58124]
 [13208 10737]]


## Model with NO-BIAS

In [53]:
#cross-validation
FNN_nob = kfold_validation(X_train_nob, y_train)

accuracy: 50.02%
accuracy: 49.98%
accuracy: 49.98%
accuracy: 50.00%
accuracy: 50.00%
50.00% (+/- 0.01%)


In [54]:
#the kfold model is giving wierd results. 
#thus, switching to this model
#I think issue stems from stratifiedKfold not working, i.e. I'm doing something wrong
FNN_nob_ = create_model(X_train_nob)
FNN_nob_.fit(X_train_nob, y_train,  epochs=100, shuffle=True, batch_size=1000, verbose=2, callbacks=[es_callback],
           class_weight=class_weights, validation_data=(X_test_nob, y_test))

Train on 471136 samples, validate on 160000 samples
Epoch 1/100
 - 2s - loss: 0.6966 - accuracy: 0.5005 - accuracy_1: 0.5005 - val_loss: 0.6980 - val_accuracy: 0.1722 - val_accuracy_1: 0.1722
Epoch 2/100
 - 2s - loss: 0.6933 - accuracy: 0.5004 - accuracy_1: 0.5004 - val_loss: 0.6944 - val_accuracy: 0.1726 - val_accuracy_1: 0.1726
Epoch 3/100
 - 2s - loss: 0.6932 - accuracy: 0.4979 - accuracy_1: 0.4979 - val_loss: 0.6940 - val_accuracy: 0.1709 - val_accuracy_1: 0.1709
Epoch 4/100
 - 2s - loss: 0.6932 - accuracy: 0.5012 - accuracy_1: 0.5012 - val_loss: 0.6961 - val_accuracy: 0.1497 - val_accuracy_1: 0.1497
Epoch 5/100
 - 2s - loss: 0.6932 - accuracy: 0.4994 - accuracy_1: 0.4994 - val_loss: 0.6926 - val_accuracy: 0.8503 - val_accuracy_1: 0.8503
Epoch 6/100
 - 2s - loss: 0.6932 - accuracy: 0.5001 - accuracy_1: 0.5001 - val_loss: 0.6963 - val_accuracy: 0.1497 - val_accuracy_1: 0.1497
Epoch 7/100
 - 2s - loss: 0.6932 - accuracy: 0.5004 - accuracy_1: 0.5004 - val_loss: 0.6943 - val_accuracy: 

In [55]:
#cross_validation on test
#train the model 10 times and predict. 
#take the average.
cvscores = []
for x in range(10):
    
    FNN_nob_.fit(X_train_nob, y_train,  epochs=100, shuffle=True, batch_size=1000, verbose=0, 
                 callbacks=[es_callback], class_weight=class_weights)
    
    scores = FNN_nob_.evaluate(X_test_nob, y_test, verbose=0)
        
    print("%s: %.2f%%" % (FNN_nob_.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
        
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

accuracy: 85.03%
accuracy: 14.97%
accuracy: 14.97%
accuracy: 85.03%
accuracy: 81.65%
accuracy: 85.03%
accuracy: 14.97%
accuracy: 14.97%
accuracy: 14.97%
accuracy: 14.97%
42.65% (+/- 33.92%)


In [56]:
#predict default with test dataset
y_pred_nob = FNN_nob_.predict_classes(X_test_nob)

print('Non-Bias classification report')
print(classification_report(y_test, y_pred_nob))
print()

print('Non-Bias confusion matrix')
print(confusion_matrix(y_test, y_pred_nob))

Non-Bias classification report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00    136055
           1       0.15      1.00      0.26     23945

    accuracy                           0.15    160000
   macro avg       0.07      0.50      0.13    160000
weighted avg       0.02      0.15      0.04    160000


Non-Bias confusion matrix


/Users/Rolf/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[     0 136055]
 [     0  23945]]


# 3. Fairness of model 

We will use three metrics to identify bias:
1. Demographic Parity 
    - Positive rate
2. Equal Opportunity 
    - True positive rate
3. Equalised Odds 
    - Combination of true positive rate and true negative rate

In [57]:
#function to split train and test dataset into 4 groups
# Minority + Default
# Minority + No-default
# Non-Minority + Default
# Non-Minority + No-default

#these groups are needed to calculate fairness metrics


def bias_analysis(X_test_df, y_test_df, df_column_names, model, bias, minority):
    """ 
    X_test_df = X_test after StandardScaler transformation
    y_test_df = y_test 
    df_columns_names = X_test dataframe before StandardScaler transformation
    model = deep learning model
    bias = 0 or 1
    minority = 0 or 1
    """
    
    #join minority variable to X and y dataframes to differentiate between minority and non-minority
    X_test_min = np.concatenate((X_test_df, 
                                 test['minority'].values.reshape(-1,1)), 
                                axis=1)
    y_test_min = np.concatenate((y_test_df.values.reshape(-1,1),
                                 test['minority'].values.reshape(-1,1)), 
                                axis=1)
    
    #column headings for dataframe that will be converted from array
    columns = np.insert(df_column_names.columns, X_test_df.shape[1], 'minority')
    
    #convert numpy array to dataframe
    df_X_test = pd.DataFrame(X_test_min, columns=columns)
    df_y_test = pd.DataFrame(y_test_min, columns=['default','minority'])
    
    #subset minority==1, minority group
    X_minority = df_X_test[df_X_test['minority']==minority].values
    y_minority = df_y_test[df_y_test['minority']==minority].values
    
    #predict y
    y_pred_minority = model.predict_classes(X_minority[:,:X_test_df.shape[1]])

    #evaluate accuracy
    loss_dl_min, accuracy_dl_min, accuracy_weighted_dl_min = model.evaluate(X_minority[:,:X_test_df.shape[1]], 
                                                                            y_minority[:,0])
    
    confusion = confusion_matrix(y_minority[:,0], y_pred_minority)
    
    if bias == 1:
        if minority == 1:
        
            print('Test loss bias minority: ' + str(round(loss_dl_min, 2)))
            print('Test accuracy bias minority: ' + str(round(accuracy_dl_min*100, 2)) + '%')

            print()

            return confusion

        else:

            print('Test loss bias non-minority: ' + str(round(loss_dl_min, 2)))
            print('Test accuracy bias non-minority: ' + str(round(accuracy_dl_min*100, 2)) + '%')

            print()

            return confusion
      
    #bias=0
    if bias == 0:
        if minority == 1:

            print('Test loss non-bias minority: ' + str(round(loss_dl_min, 2)))
            print('Test accuracy non-bias minority: ' + str(round(accuracy_dl_min*100, 2)) + '%')

            print()

            return confusion

        else:
            
            print('Test loss non-bias non-minority: ' + str(round(loss_dl_min, 2)))
            print('Test accuracy non-bias non-minority: ' + str(round(accuracy_dl_min*100, 2)) + '%')

            print()

            return confusion
        
    if bias==2:
                    
            print('Test loss downsampled non-minority: ' + str(round(loss_dl_min, 2)))
            print('Test accuracy downsampled non-minority: ' + str(round(accuracy_dl_min*100, 2)) + '%')

            print()

            return confusion

In [58]:
#function to reduce confusion regarding which variales to use in bias_analysis function

def bias_check(bias, minority):
    """
    bias = 0 or 1
        bias==1 means dataset is biased
    minority = 0 or 1
        minority==1 means minority is analysed
        
    bias ==2 
        when train dataset has been downsampled

    """
    
    if bias == 1:
        result = bias_analysis(X_test_bias, y_test, X_te_bias, FNN_bias, bias=bias, minority=minority)
        return result
    
    if bias == 0:
        result = bias_analysis(X_test_nob, y_test, X_te_nob, FNN_nob, bias=bias, minority=minority)
        return result
    
    if bias == 2:
        result = bias_analysis(X_test_bias, y_test, X_te_bias, FNN_ds, bias=bias, minority=minority)

### Calculate confusion martrix

Confusion matrix is needed to calculate fp, tn, fn, tp, which are needed to calculate fairness metrics

In [59]:
#confusion matrix for BIAS and Minority
confusion_bias_minority = bias_check(bias=1, minority=1)

80348/80348 [==============================] - 2s 19us/step
Test loss bias minority: 10.13
Test accuracy bias minority: 27.03%



In [60]:
#confusion matrix for BIAS and Non-Minority
confusion_bias_non_minority = bias_check(bias=1, minority=0)

79652/79652 [==============================] - 2s 23us/step
Test loss bias non-minority: 3.27
Test accuracy bias non-minority: 84.05%



In [61]:
#confusion matrix for Non-BIAS and Minority
confusion_non_bias_minority = bias_check(bias=0, minority=1)

80348/80348 [==============================] - 2s 25us/step
Test loss non-bias minority: 0.69
Test accuracy non-bias minority: 84.97%



In [62]:
#confusion matrix for Non-Bias and Non-Minority
confusion_non_bias_non_minority = bias_check(bias=0, minority=0)

79652/79652 [==============================] - 2s 30us/step
Test loss non-bias non-minority: 0.69
Test accuracy non-bias non-minority: 85.1%



# Demographic Parity

Demographic parity is achieved when the positive rate in minority and non-minority are similar. Positive in our case means default. We assume a 5% difference between minority and non-minority to mean that they are similar. 

According to Demographic Parity:
- **Bias** dataset is not fair.

- **Non-Bias** dataset is fair.

Really good article explaining fairness measures in ML
https://towardsdatascience.com/how-to-define-fairness-to-detect-and-prevent-discriminatory-outcomes-in-machine-learning-ef23fd408ef2

<img src=images/demographic_parity.png>

In [63]:
#function to calculate demographic parity

def demographic_parity(result, bias=0, minority=0):
    """
    bias = 0 or 1
        bias==1 means dataset is biased
    minority = 0 or 1
        minority==1 means minority is analysed

    """
    
    tn, fp, fn, tp = result.ravel()
    #tn, fp, fn, tp = bias_check(bias, minority).ravel()
    
    temp = (fp + tp) / (tn + fp + fn + tp)
    demo_parity = round(temp*100, 2) 
    
    if bias == 1:
        if minority == 0:
            print('Demographic parity for bias non-minority: ' + str(demo_parity) + '%')

        
        else: 
            print('Demographic parity for bias minority: ' + str(demo_parity) + '%')
   
    if bias == 0:
        if minority == 0:
            print('Demographic parity for non-bias non-minority: ' + str(demo_parity) + '%')    
            
        else: 
            print('Demographic parity for non-bias minority: ' + str(demo_parity) + '%')  
            
    if bias == 2:
        if minority == 0:
            print('Demographic parity for downsampled non-minority: ' + str(demo_parity) + '%')    
            
        else: 
            print('Demographic parity for downsampled minority: ' + str(demo_parity) + '%')  
            

In [64]:
#BIAS
demographic_parity(confusion_bias_minority, bias=1, minority=1)
demographic_parity(confusion_bias_non_minority, bias=1, minority=0)

Demographic parity for bias minority: 83.76%
Demographic parity for bias non-minority: 1.96%


In [65]:
#Non-BIAS
demographic_parity(confusion_non_bias_minority, bias=0, minority=1)
demographic_parity(confusion_non_bias_non_minority, bias=0, minority=0)

Demographic parity for non-bias minority: 0.0%
Demographic parity for non-bias non-minority: 0.0%


# Equal Opportunity

Equal Opportunity is achieved when the true positive rate in minority and non-minority are similar. Positive in our case means default. We assume a 5% difference between minority and non-minority to mean that they are similar. 

According to Equal Opportunity:
- **Bias** dataset is not fair.

- **Non-Bias** dataset is fair.

<img src=images/equal_opportunity.png>

In [66]:
#function to calculate equal opportunity
def equal_opportunity(result, bias=0, minority=0):
    """
    bias = 0 or 1
        bias==1 means dataset is biased
    minority = 0 or 1
        minority==1 means minority is analysed

    """
    
    tn, fp, fn, tp = result.ravel()
    #tn, fp, fn, tp = bias_check(bias, minority).ravel()
    
    temp = tp / (fn + tp)
    equal_opportunity = round(temp*100, 2) 
    
    if bias == 1:
        if minority == 0:
            print('Equal opportunity for bias non-minority: ' + str(equal_opportunity) + '%')

        
        else: 
            print('Equal opportunity for bias minority: ' + str(equal_opportunity) + '%')

    
    if bias == 0:
        if minority == 0:
            print('Equal opportunity for non-bias non-minority: ' + str(equal_opportunity) + '%')

        
        else: 
            print('Equal opportunity for non-bias minority: ' + str(equal_opportunity) + '%')

        
    if bias==2:
        if minority == 0:
            print('Equal opportunity for downsampled non-minority: ' + str(equal_opportunity) + '%')

        
        else: 
            print('Equal opportunity for downsampled minority: ' + str(equal_opportunity) + '%')


In [67]:
#BIAS
equal_opportunity(confusion_bias_minority, bias=1, minority=1)
equal_opportunity(confusion_bias_non_minority, bias=1, minority=0)

Equal opportunity for bias minority: 85.91%
Equal opportunity for bias non-minority: 3.05%


In [68]:
#Non-BIAS
equal_opportunity(confusion_non_bias_minority, bias=0, minority=1)
equal_opportunity(confusion_non_bias_non_minority, bias=0, minority=0)

Equal opportunity for non-bias minority: 0.0%
Equal opportunity for non-bias non-minority: 0.0%


# Equalised Odds

Equalised Odds is achieved when the true positive rate and the true negative rate in minority and non-minority are similar. Positive in our case means default. We assume a 5% difference between minority and non-minority to mean that they are similar. 

According to Equalised Odds:
- **Bias** dataset is not fair.

- **Non-Bias** dataset is fair.

<img src=images/equalised_odds.png>

In [69]:
#function to calculate equlaised odds
def equalised_odds(result, bias=0, minority=0):
    """
    bias = 0 or 1
        bias==1 means dataset is biased
    minority = 0 or 1
        minority==1 means minority is analysed

    """
    
    tn, fp, fn, tp = result.ravel()
    #tn, fp, fn, tp = bias_check(bias, minority).ravel()
    
    temp = tp / (fn + tp)
    true_positive_rate = round(temp*100, 2)
    
    temp = tn / (fp + tn)
    true_negative_rate = round(temp*100, 2)
    
    if bias == 1:
        if minority == 0:
            print()
            print('True positive rate for bias non-minority: ' + str(true_positive_rate) + '%')
            print('True negative rate for bias non-minority: ' + str(true_negative_rate) + '%')

        
        else: 
            print()
            print('True positive rate for bias minority: ' + str(true_positive_rate) + '%')
            print('True negative rate for bias minority: ' + str(true_negative_rate) + '%')

    
    if bias == 0:
        if minority == 0:
            print()
            print('True positive rate for non-bias non-minority: ' + str(true_positive_rate) + '%')
            print('True negative rate for non-bias non-minority: ' + str(true_negative_rate) + '%')

        
        else: 
            print()
            print('True positive rate for non-bias minority: ' + str(true_positive_rate) + '%')
            print('True negative rate for non-bias minority: ' + str(true_negative_rate) + '%')

        
    if bias==2:
        if minority == 0:
            print()
            print('True positive rate for downsampled non-minority: ' + str(true_positive_rate) + '%')
            print('True negative rate for downsampled non-minority: ' + str(true_negative_rate) + '%')

        
        else: 
            print()
            print('True positive rate for downsampled minority: ' + str(true_positive_rate) + '%')
            print('True negative rate for downsampled minority: ' + str(true_negative_rate) + '%')        

In [70]:
#BIAS
equalised_odds(confusion_bias_minority, bias=1, minority=1)
equalised_odds(confusion_bias_non_minority, bias=1, minority=0)

#Non-BIAS
equalised_odds(confusion_non_bias_minority, bias=0, minority=1)
equalised_odds(confusion_non_bias_non_minority, bias=0, minority=0)


True positive rate for bias minority: 85.91%
True negative rate for bias minority: 16.62%

True positive rate for bias non-minority: 3.05%
True negative rate for bias non-minority: 98.23%

True positive rate for non-bias minority: 0.0%
True negative rate for non-bias minority: 100.0%

True positive rate for non-bias non-minority: 0.0%
True negative rate for non-bias non-minority: 100.0%


# Results Discussion

Overall, the bias dataset is biased (not fair) since none of the above metrics (demographic parity, equal opportunity, or equalised odds) are within 5% difference. 

The non-bias dataset is deemed to be fair according to the above metrics (demographic parity, equal opportunity, and equalised odds) since all results are within 5% difference. This proves our hypothesis that removing the biased variables (ZIP, rent, job_stability and occupation) does remove bias, i.e. makes the model fairer. 

However, removing four variables might reduce accuracy (lose signal) and not be in a business's interest. Therefore, we will try another approach, down-sampling, to reduce bias and keep the four variables.

# 4. Down-sampled Train Dataset

With down-sampling, we aim to retain the four variables (ZIP, rent, job_stability and occupation), which are the cause of bias in our train dataset, and at the same time reduce bias.

We hope to reduce bias by down-sampling the train dataset to have 50% positive rate (default rate) in minority and non-minority group. To achieve this we will down-sample the following to 250 cases (we use 250 cases sine it is the small common denominator among the four groups):
- Minority and default = 250 cases
- Minority and non-default = 250 cases
- Non-Minority and default = 250 cases
- Non-Minority and default = 250 cases

Overall, we will have a down-sampled train dataset with 1'000 cases (rows). 

Inspired by this website for downsampling:
https://elitedatascience.com/imbalanced-classes

In [71]:
#downsampling


# Separate majority and minority classes
df_non_minority_0 = train[(train['minority']==0) & (train['default']==0)]
df_non_minority_1 = train[(train['minority']==0) & (train['default']==1)]

df_minority_0 = train[(train['minority']==1) & (train['default']==0)]
df_minority_1 = train[(train['minority']==1) & (train['default']==1)]
 

# Downsample non_minority, default == 0
df_non_minority_0_downsampled = resample(df_non_minority_0, 
                                         replace=False,    # sample without replacement
                                         n_samples=250,     # to match minority class
                                         random_state=seed) # reproducible results

# Downsample non_minority, default == 1
df_non_minority_1_downsampled = resample(df_non_minority_1, 
                                         replace=False,    # sample without replacement
                                         n_samples=250,     # to match minority class
                                         random_state=seed) # reproducible results

# Downsample minority, default == 1
df_minority_1_downsampled = resample(df_minority_1, 
                                         replace=False,    # sample without replacement
                                         n_samples=250,     # to match minority class
                                         random_state=seed) # reproducible results

# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_non_minority_0_downsampled,
                            df_non_minority_1_downsampled,
                            df_minority_0,
                            df_minority_1_downsampled
                           ])
 
# Display new class counts
print(df_downsampled['minority'].value_counts())

#Display distribution for minority, default
df_downsampled.groupby(["minority", "default"])["minority"].count()

1    500
0    500
Name: minority, dtype: int64


minority  default
0         0          250
          1          250
1         0          250
          1          250
Name: minority, dtype: int64

In [72]:
#create dataframe with independent variables (X) and indepedent variable (y)
X_ds = df_downsampled.drop(['default', 'minority', 'sex'], axis=1)
y_downsampled = df_downsampled['default']

#Scale X train/test bias
scaler = StandardScaler()
X_fit_bias_ds = scaler.fit(X_ds)

X_downsampled = X_fit_bias_ds.transform(X_ds)

## LogisticRegression - Down-sampled

In [73]:
#Logistic Regression

lr_ds = LogisticRegression(solver='lbfgs')
lr_ds.fit(X_downsampled, y_downsampled)

y_pred_ds = lr_ds.predict(X_test_bias)
accuracy_lr_ds = accuracy_score(y_pred_ds, y_test)


#print accuracy
print('LogisticRegression accuracy with downsampled dataset: ' + str(round(accuracy_lr_ds*100,2)) + '%')
print()

#print classification report
print(classification_report(y_test, y_pred_ds))

LogisticRegression accuracy with downsampled dataset: 36.82%

              precision    recall  f1-score   support

           0       0.84      0.32      0.46    136055
           1       0.14      0.65      0.24     23945

    accuracy                           0.37    160000
   macro avg       0.49      0.48      0.35    160000
weighted avg       0.73      0.37      0.43    160000



## Deep learning - Down-sampled

In [74]:
#train deep learning model with 5-kfold validation
FNN_ds = kfold_validation(X_downsampled, y_downsampled)

accuracy: 87.50%
accuracy: 89.00%
accuracy: 89.00%
accuracy: 87.00%
accuracy: 86.50%
87.80% (+/- 1.03%)


In [75]:
#the kfold model is giving wierd results. 
#thus, switching to this model
#I think issue stems from stratifiedKfold not working, i.e. I'm doing something wrong
FNN_ds = create_model(X_downsampled)
FNN_ds.fit(X_downsampled, y_downsampled, epochs=100, batch_size=1000, shuffle=True, verbose=2, callbacks=[es_callback],
           class_weight=class_weights, validation_data=(X_test_bias, y_test))

Train on 1000 samples, validate on 160000 samples
Epoch 1/100
 - 1s - loss: 0.7180 - accuracy: 0.5380 - accuracy_1: 0.5380 - val_loss: 0.9340 - val_accuracy: 0.1676 - val_accuracy_1: 0.1676
Epoch 2/100
 - 0s - loss: 0.7147 - accuracy: 0.5350 - accuracy_1: 0.5350 - val_loss: 0.9200 - val_accuracy: 0.1688 - val_accuracy_1: 0.1688
Epoch 3/100
 - 0s - loss: 0.7175 - accuracy: 0.5160 - accuracy_1: 0.5160 - val_loss: 0.9075 - val_accuracy: 0.1697 - val_accuracy_1: 0.1697
Epoch 4/100
 - 0s - loss: 0.6973 - accuracy: 0.5450 - accuracy_1: 0.5450 - val_loss: 0.8950 - val_accuracy: 0.1715 - val_accuracy_1: 0.1715
Epoch 5/100
 - 0s - loss: 0.6858 - accuracy: 0.5860 - accuracy_1: 0.5860 - val_loss: 0.8831 - val_accuracy: 0.1734 - val_accuracy_1: 0.1734
Epoch 6/100
 - 0s - loss: 0.6685 - accuracy: 0.6000 - accuracy_1: 0.6000 - val_loss: 0.8720 - val_accuracy: 0.1759 - val_accuracy_1: 0.1759
Epoch 7/100
 - 0s - loss: 0.6708 - accuracy: 0.5770 - accuracy_1: 0.5770 - val_loss: 0.8614 - val_accuracy: 0.

Epoch 59/100
 - 0s - loss: 0.3316 - accuracy: 0.8900 - accuracy_1: 0.8900 - val_loss: 0.8299 - val_accuracy: 0.4850 - val_accuracy_1: 0.4850
Epoch 60/100
 - 0s - loss: 0.3340 - accuracy: 0.8680 - accuracy_1: 0.8680 - val_loss: 0.8365 - val_accuracy: 0.4846 - val_accuracy_1: 0.4846
Epoch 61/100
 - 0s - loss: 0.3288 - accuracy: 0.8910 - accuracy_1: 0.8910 - val_loss: 0.8444 - val_accuracy: 0.4832 - val_accuracy_1: 0.4832
Epoch 62/100
 - 0s - loss: 0.3185 - accuracy: 0.8750 - accuracy_1: 0.8750 - val_loss: 0.8521 - val_accuracy: 0.4821 - val_accuracy_1: 0.4821
Epoch 63/100
 - 0s - loss: 0.3159 - accuracy: 0.8840 - accuracy_1: 0.8840 - val_loss: 0.8612 - val_accuracy: 0.4798 - val_accuracy_1: 0.4798
Epoch 64/100
 - 0s - loss: 0.3088 - accuracy: 0.8810 - accuracy_1: 0.8810 - val_loss: 0.8690 - val_accuracy: 0.4786 - val_accuracy_1: 0.4786
Epoch 65/100
 - 0s - loss: 0.3098 - accuracy: 0.8720 - accuracy_1: 0.8720 - val_loss: 0.8765 - val_accuracy: 0.4781 - val_accuracy_1: 0.4781
Epoch 66/100


In [76]:
#accuracy of downsampled dataset on test dataset
y_pred_dl_ds = FNN_ds.predict_classes(X_test_bias, verbose=0)

accuracy_dl_ds = accuracy_score(y_pred_dl_ds, y_test)

#print accuracy
print('Deep Learning accuracy with downsampled dataset: ' + str(round(accuracy_dl_ds*100,2)) + '%')
print()

print(classification_report(y_test, y_pred_dl_ds))

print()

print('Down-sampled confusion matrix')
print(confusion_matrix(y_test, y_pred_nob))

Deep Learning accuracy with downsampled dataset: 52.75%

              precision    recall  f1-score   support

           0       0.91      0.49      0.64    136055
           1       0.20      0.71      0.31     23945

    accuracy                           0.53    160000
   macro avg       0.55      0.60      0.48    160000
weighted avg       0.80      0.53      0.59    160000


Down-sampled confusion matrix
[[     0 136055]
 [     0  23945]]


## Fairness - Identifying Bias

In [77]:
#calculate confusion matrix
confusion_ds_minority = bias_analysis(X_test_bias, y_test, X_te_bias, FNN_ds, bias=2, minority=1)

80348/80348 [==============================] - 1s 17us/step
Test loss downsampled non-minority: 0.79
Test accuracy downsampled non-minority: 62.98%



In [78]:
confusion_ds_non_minority = bias_analysis(X_test_bias, y_test, X_te_bias, FNN_ds, bias=2, minority=0)

79652/79652 [==============================] - 1s 17us/step
Test loss downsampled non-minority: 1.23
Test accuracy downsampled non-minority: 42.43%



### Demographic Parity - Downsampled

In [79]:
#demographic parity for downsampled minority
demographic_parity(confusion_ds_minority, bias=2, minority=1)

Demographic parity for downsampled minority: 41.27%


In [80]:
#demographic parity for downsampled non-minority
demographic_parity(confusion_ds_non_minority, bias=2, minority=0)

Demographic parity for downsampled non-minority: 66.1%


### Equal Opportunity - Downsampled

In [81]:
#equal opportunity for downsampled minority. 
equal_opportunity(confusion_ds_minority, bias=2, minority=1)

Equal opportunity for downsampled minority: 64.12%


In [82]:
#equal opportunity for downsampled non-minority. 
equal_opportunity(confusion_ds_non_minority, bias=2, minority=0)

Equal opportunity for downsampled non-minority: 78.62%


### Equalised Odds - Downsampled

In [83]:
#equalised odds for downsampled minority
equalised_odds(confusion_ds_minority, bias=2, minority=1)


True positive rate for downsampled minority: 64.12%
True negative rate for downsampled minority: 62.78%


In [84]:
#equalised odds for donwsampled non-minority
equalised_odds(confusion_ds_non_minority, bias=2, minority=0)


True positive rate for downsampled non-minority: 78.62%
True negative rate for downsampled non-minority: 36.09%


# Results Discussion - Down-Sampled

According to demographic parity, equal opportunity and equalised odds, the down-sampled reduces bias compared to the bias dataset, however, bias still exists in the down-sampled dataset. None of the metrics have a differences between minority and non-minority of less than 5%. 

Down-sampling a dataset from +450'000 rows to only 1'000 is drastic and not ideal. Therefore, we will try to use disparate impact remover to remove the bias from the train dataset. 

# 5. Disparate Impact Remover

Disparate impact remover on bias dataset did not work, since it only predicts 1s at an accuracy of 15%. 

Notebook on how to remove bias in a variable: https://nbviewer.jupyter.org/github/srnghn/bias-mitigation-examples/blob/master/Bias%20Mitigation%20with%20Disparate%20Impact%20Remover.ipynb

In [85]:
#create dataset and prepare for disparate impact remover
train_BLD = BinaryLabelDataset(favorable_label='1',
                                unfavorable_label='0',
                                df=train,
                                label_names=['default'],
                                protected_attribute_names=['minority'],
                                unprivileged_protected_attributes=['1'])
test_BLD = BinaryLabelDataset(favorable_label='1',
                                unfavorable_label='0',
                                df=test,
                                label_names=['default'],
                                protected_attribute_names=['minority'],
                                unprivileged_protected_attributes=['1'])

In [86]:
#transform dataset to remove disparate impact
di = DisparateImpactRemover(repair_level=1.0)
rp_train = di.fit_transform(train_BLD)
rp_test = di.fit_transform(test_BLD)

In [87]:
#train deep learning model
FNN_rp = create_model(rp_train.features[:, 2:]) #first two columns dropped since sex and minority
FNN_rp.fit(rp_train.features[:, 2:], rp_train.labels, epochs=100, batch_size=1000, shuffle=True, verbose=2, 
           callbacks=[es_callback], validation_data=(rp_test.features[:, 2:], y_test))

Train on 471136 samples, validate on 160000 samples
Epoch 1/100
 - 2s - loss: 646.5594 - accuracy: 0.4995 - accuracy_1: 0.4995 - val_loss: 0.6987 - val_accuracy: 0.1497 - val_accuracy_1: 0.1497
Epoch 2/100
 - 2s - loss: 4.1756 - accuracy: 0.5002 - accuracy_1: 0.5002 - val_loss: 0.6927 - val_accuracy: 0.8503 - val_accuracy_1: 0.8503
Epoch 3/100
 - 2s - loss: 2.1113 - accuracy: 0.5000 - accuracy_1: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.8503 - val_accuracy_1: 0.8503
Epoch 4/100
 - 2s - loss: 1.5326 - accuracy: 0.4990 - accuracy_1: 0.4990 - val_loss: 0.6940 - val_accuracy: 0.1497 - val_accuracy_1: 0.1497
Epoch 5/100
 - 2s - loss: 1.1074 - accuracy: 0.4997 - accuracy_1: 0.4997 - val_loss: 0.6938 - val_accuracy: 0.1497 - val_accuracy_1: 0.1497
Epoch 6/100
 - 2s - loss: 0.8910 - accuracy: 0.5002 - accuracy_1: 0.5002 - val_loss: 0.6932 - val_accuracy: 0.1497 - val_accuracy_1: 0.1497
Epoch 7/100
 - 2s - loss: 0.8523 - accuracy: 0.4990 - accuracy_1: 0.4990 - val_loss: 0.6929 - val_accuracy

In [88]:
#predict labels
di_preds = FNN_rp.predict(rp_test.features[:, 2:])

In [89]:
#create dataframe with labels and minority
di_pred_np = np.concatenate((test[['minority']],
                                di_preds), 
                                axis=1)
di_pred_df = pd.DataFrame(di_pred_np, columns=['minority', 'preds'])
di_pred_df.head()

,minority,preds
0,1.0,0.501546
1,0.0,0.501546
2,0.0,0.501546
3,1.0,0.501546
4,1.0,0.501546


In [90]:
#only 0.5s are predicted
#input in Sigmoid function is too close to 0, thus, 0.5 returned. 
di_pred_df['preds'].mean()

0.5015462636947632

In [91]:
#use model not trained disparate impact remover
#predict using test transformed by disparate impact remover
y_pred_dl_bias_di = FNN_rp.predict_classes(rp_test.features[:, 2:], verbose=0)

accuracy_dl_bias_di = accuracy_score(y_pred_dl_bias_di, y_test)

#print accuracy
print('Deep Learning accuracy after disparate impact remover on bias dataset: ' + 
      str(round(accuracy_dl_bias_di*100,2)) + '%')
print()

print(classification_report(y_test, y_pred_dl_bias_di))

print()

print(confusion_matrix(y_test, y_pred_dl_bias_di))

Deep Learning accuracy after disparate impact remover on bias dataset: 14.97%

              precision    recall  f1-score   support

           0       0.00      0.00      0.00    136055
           1       0.15      1.00      0.26     23945

    accuracy                           0.15    160000
   macro avg       0.07      0.50      0.13    160000
weighted avg       0.02      0.15      0.04    160000




/Users/Rolf/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[     0 136055]
 [     0  23945]]


## Disparate Impact Remover - Down-sampled

The results for disparate impact remover on bias dataset are disappointing. Therefore, we apply disparate impact remover to down-sampled dataset.

On the down-sampled dataset, after applying disparate impact remover, only 0s (no default) are predicted for an accuracy of 85%. However, a model that only predicts 0s is not useful i.e. the model is not generalising.

In [92]:
#create disparate impact remover
#preparation for next code cell below
train_BLD_ds = BinaryLabelDataset(favorable_label='1',
                                unfavorable_label='0',
                                df=df_downsampled,
                                label_names=['default'],
                                protected_attribute_names=['minority'],
                                unprivileged_protected_attributes=['1'])

In [93]:
#fit disaparate impact remover on train
di = DisparateImpactRemover(repair_level=1.0)
rp_train_ds = di.fit_transform(train_BLD_ds)

In [94]:
#downsampled dataset with disparate impact remover
FNN_rp_ds = create_model(rp_train.features[:, 2:]) #first two columns dropped since sex and minority
FNN_rp_ds.fit(rp_train_ds.features[:, 2:], rp_train_ds.labels, epochs=100, batch_size=1000, 
           shuffle=True, verbose=2, callbacks=[es_callback], validation_data=(rp_test.features[:, 2:], y_test))

Train on 1000 samples, validate on 160000 samples
Epoch 1/100
 - 1s - loss: 8448.7256 - accuracy: 0.4880 - accuracy_1: 0.4880 - val_loss: 5989.8347 - val_accuracy: 0.1497 - val_accuracy_1: 0.1497
Epoch 2/100
 - 0s - loss: 7789.2539 - accuracy: 0.4980 - accuracy_1: 0.4980 - val_loss: 5830.0907 - val_accuracy: 0.1497 - val_accuracy_1: 0.1497
Epoch 3/100
 - 0s - loss: 7395.8545 - accuracy: 0.5180 - accuracy_1: 0.5180 - val_loss: 5601.0810 - val_accuracy: 0.1497 - val_accuracy_1: 0.1497
Epoch 4/100
 - 0s - loss: 7989.7739 - accuracy: 0.4940 - accuracy_1: 0.4940 - val_loss: 5301.1136 - val_accuracy: 0.1497 - val_accuracy_1: 0.1497
Epoch 5/100
 - 0s - loss: 7504.8662 - accuracy: 0.4990 - accuracy_1: 0.4990 - val_loss: 5002.4826 - val_accuracy: 0.1497 - val_accuracy_1: 0.1497
Epoch 6/100
 - 0s - loss: 6991.9814 - accuracy: 0.5180 - accuracy_1: 0.5180 - val_loss: 4711.8496 - val_accuracy: 0.1497 - val_accuracy_1: 0.1497
Epoch 7/100
 - 0s - loss: 6884.0806 - accuracy: 0.5300 - accuracy_1: 0.530

In [95]:
#predict default rate
di_preds_ds = FNN_rp_ds.predict(rp_test.features[:, 2:])

#concat predictions with minority
di_pred_np_ds = np.concatenate((test[['minority']],
                                di_preds_ds), 
                                axis=1)

#convert to dataframe
di_pred_df_ds = pd.DataFrame(di_pred_np_ds, columns=['minority', 'preds'])
di_pred_df_ds.head()

,minority,preds
0,1.0,0.0
1,0.0,0.0
2,0.0,0.0
3,1.0,0.0
4,1.0,0.0


In [96]:
#only 1s are predicted
di_pred_df_ds['preds'].mean()

0.0

In [97]:
#use model not trained disparate impact remover
#predict using test transformed by disparate impact remover
y_pred_dl_ds_di = FNN_rp_ds.predict_classes(rp_test.features[:, 2:], verbose=0)

accuracy_dl_ds_di = accuracy_score(y_pred_dl_ds_di, y_test)

#print accuracy
print('Deep Learning accuracy after disparate impact remover on downsampled dataset: ' + 
      str(round(accuracy_dl_ds_di*100,2)) + '%')
print()

print(classification_report(y_test, y_pred_dl_ds_di))

print()
print(confusion_matrix(y_test, y_pred_dl_ds_di))

Deep Learning accuracy after disparate impact remover on downsampled dataset: 85.03%

              precision    recall  f1-score   support

           0       0.85      1.00      0.92    136055
           1       0.00      0.00      0.00     23945

    accuracy                           0.85    160000
   macro avg       0.43      0.50      0.46    160000
weighted avg       0.72      0.85      0.78    160000




/Users/Rolf/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[136055      0]
 [ 23945      0]]
